<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Coba_Logic_Rekomendasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymssql

import pymssql
import pandas as pd
import numpy as np
import gspread
import seaborn as sns
import matplotlib.pyplot as plt
from gspread_dataframe import get_as_dataframe
import gspread
from google.colab import auth
from google.auth import default
import warnings

warnings.filterwarnings("ignore")
from google.colab import drive

#Mount Google Drive
drive.mount('/content/drive')

# Autentikasi Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.1 MB/s eta 0:00:00
Mounted at /content/drive


# Load Dataset

In [2]:
url_dokter_survey = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NIOYmajDgkSY7lfKNH_wHRz0I80eeZX2OWhNtAdMw-8/edit?gid=0#gid=0')
url_database_ethical = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Qwurf3wlLDbrhWNB0XX1CgpK7GuHqCHZoAlQK_GJqek/edit?usp=sharing')
url_data_score_dokter = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wIe9UOG8F6aNy1PF1UzGNQnkrcCm6DAD5zjUEnMbUo8/edit?usp=sharing')

In [4]:
#Koneksi ke Database
connection = pymssql.connect(server='35.187.243.174', user='tim_powerbi',
                             password='dGltcG93ZXJiaQ', database='mkt_insight', port = '1433')


#Menarik data kode PI dan item kode
query_1 = """
SELECT DISTINCT KodePI, [Item Kode]
FROM mkt_insight.dbo.vw_DIR10001B
WHERE KodePI IS NOT NULL
  AND KodePI <> ''
  AND (Periode LIKE '%2023%' OR Periode LIKE '%2024%')
"""


#Menarik data yang menghasilkan item kode, nama produk, zat aktif, dan total penjualan
query_2 = """
SELECT z.[Nama Produk], s.[Item Kode], z.[Zat Aktif], SUM(s.[Value Sales]) AS [Total Penjualan]
FROM mkt_insight.dbo.DIR10001B s
INNER JOIN mkt_insight.dbo.ListAllProductComplete z ON s.[Item Kode]= z.[Kode Produk PI]
GROUP BY s.[Item Kode], z.[Nama Produk], z.[Zat Aktif]"""


#Menarik data nama outlet dan kode PI
query_3 = """
SELECT DISTINCT(KodePI), NamaOutlet, Propinsi, Alamat
FROM mkt_insight.dbo.Struktur_Marketing_PI
"""

# Menarik Data PSSP
query_4 = """
SELECT * FROM mkt_insight.dbo.PSSP_PIdanPML
"""

'''
data_1 = pd.DataFrame(pd.read_sql_query(query_1, connection))
data_2 = pd.DataFrame(pd.read_sql_query(query_2, connection))'''

# Dataset untuk mencocokan nama outlet terdaftar
data_struktur_PI = pd.DataFrame(pd.read_sql_query(query_3, connection))

#Dataset produk dan zat aktif perusahaan
data_produk_zat_aktif = pd.DataFrame(pd.read_sql_query(query_2, connection))

#Dataset PSSP
data_pssp = pd.DataFrame(pd.read_sql_query(query_4, connection))

#Dataset standarisasi
data_standarisasi = pd.DataFrame(pd.read_sql_query(query_1, connection))

# Menutup koneksi
connection.close()

In [40]:
#Dataset berisi nama dokter dan kode dokter yang terdata di Database
data_dokter = get_as_dataframe(url_dokter_survey.worksheet('DB Dokter'), evaluate_formulas=True)
data_dokter = data_dokter.loc[:, ~data_dokter.columns.str.contains('^Unnamed')]

#Dataset produk pharos beserta zat aktifnya
data_produk_pharos = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI (Final).xlsx', dtype={'Kode Produk PI':str})

#Dataset produk ethical yang meliputi nama produk, spesialis, dan alasan penggunaan
data_produk_ethical = get_as_dataframe(url_database_ethical.worksheet('Kategori Produk'), evaluate_formulas=True, dtype={'Kd Item': str, ' ':str})
data_produk_ethical = data_produk_ethical.loc[:, ~data_produk_ethical.columns.str.contains('^Unnamed')]

#Dataset score dokter. Menggunakan kolom Pelunasan AL
data_score_dokter =  get_as_dataframe(url_data_score_dokter.worksheet('Sheet1'), evaluate_formulas=True)
data_score_dokter = data_score_dokter.loc[:, ~data_score_dokter.columns.str.contains('^Unnamed')]

data_survey_final =  pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk Survey (Final).xlsx')

# Cleaning Data

In [64]:
data_survey_zat = data_survey_final.copy()

#Cleaning data survey
data_survey_zat.iloc[:, 4:16] =data_survey_zat.iloc[:, 4:16].fillna(0)
data_survey_zat = data_survey_zat[data_survey_zat['Pack Molecule String'].isnull() == False]
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_survey_zat['Pack Molecule String'] = data_survey_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [65]:
data_pi_zat = data_produk_pharos.copy()

#Cleaning data pi
data_pi_zat = data_pi_zat[data_pi_zat['Pack Molecule String'].isnull() == False]
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('\n', '', regex=False).str.replace(', ', ',', regex=False)
data_pi_zat['Pack Molecule String'] = data_pi_zat['Pack Molecule String'].str.replace('+', ',', regex=False)

In [66]:
#Memfilter data yang memiliki kode outlet dan kode dokter
data_dokter = data_dokter[(data_dokter['Kd Outlet'] != '-') & (data_dokter['Kd Dokter\n(DB Pharos)'] != '-')]
data_dokter = data_dokter.drop_duplicates()

#Menggabungkan data survey dengan data dokter berdasarkan kolom 'Doctor Name'
data_merged = pd.merge(data_survey_zat, data_dokter[['Kd Outlet','Doctor Name','Kd Dokter\n(DB Pharos)','Doctor Name \n(DB Pharos)','Kd Spesialis \n(DB Pharos)','Nama Spesialis\n(DB Pharos)']], on='Doctor Name', how='left')

data_merged = data_merged[data_merged['Kd Dokter\n(DB Pharos)'].isnull() == False]

#Membuat kolom Total
data_merged["Total"] = data_merged.iloc[:, 4:16].sum(axis=1)
data_merged = data_merged[data_merged['Total'] > 0]

# Menghapus baris yang mengandung kata 'retur' di nama produk
data_merged = data_merged[~data_merged['Product Name without #'].str.contains('retur', case=False, na=False)]

#Mengubah beberapa nama kolom
data_merged = data_merged.rename(columns = {
    'Kd Outlet' : "Kode RS",
    'Kd Dokter\n(DB Pharos)' : "Kode Dokter",
    'Doctor Name \n(DB Pharos)':"Nama Dokter",
    'Hospital Name':'Nama Rumah Sakit',
    'Kd Spesialis \n(DB Pharos)' : 'Kode Spesialis',
    'Nama Spesialis\n(DB Pharos)' : 'Nama Spesialis'
})

#Melakukan groupby berdasarkan kode rs, kode dokter, zat aktif
data_zat_aktif_dokter = data_merged.groupby(['Kode RS','Kode Dokter', 'Kode Spesialis','Pack Molecule String','Dosage Form'])['Total'].sum().reset_index()
data_zat_aktif_dokter

,Kode RS,Kode Dokter,Kode Spesialis,Pack Molecule String,Dosage Form,Total
0,A1005681,A1000684,003,"AGAR,PARAFFIN OIL,PHENOLPHTHALEIN",syr,1.0
1,A1005681,A1000684,003,ATTAPULGITE,tab,18.0
2,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",pwd,1.0
3,A1005681,A1000684,003,BUPIVACAINE,inj,1.0
4,A1005681,A1000684,003,"CALCIUM,DL-LACTIC ACID,POTASSIUM,SODIUM",inf,3.0
...,...,...,...,...,...,...
76643,I1001005,Q1002060,011,METHYLPREDNISOLONE,tab,4.0
76644,I1001005,Q1002060,011,OFLOXACIN,drp,17.0
76645,I1001005,Q1002060,011,PARACETAMOL,syr,3.0
76646,I1001005,Q1002060,011,"PSEUDOEPHEDRINE,TRIPROLIDINE",tab,1547.0


In [78]:
data_pi_merged = pd.merge(data_pi_zat,data_produk_zat_aktif[['Item Kode','Total Penjualan']],
                           left_on = 'Kode Produk PI',
                           right_on = 'Item Kode',
                           how = 'left')

data_pi_merged = data_pi_merged.drop(['Best Match IMS Pack','Best Similarity Score','Item Kode'], axis = 1)
data_pi_merged

,Kode Produk PI,Nama Produk,Pack Molecule String,Dosage Form,Kode Form,Jenis Form,Total Penjualan
0,000058,FAKTU OINTMENT 20 G,"CINCHOCAINE,POLICRESULEN",oint,G,TOPIC DERMATOL HAEMOR EXTERN,8.090191e+10
1,000069,FAKTU SUPPOSITORIA,"CINCHOCAINE,POLICRESULEN",suppos,D,Rectal,4.732396e+10
2,000070,NEBACETIN OINTMENT 5 G,"BACITRACIN,NEOMYCIN",oint,G,TOPIC DERMATOL HAEMOR EXTERN,2.848903e+11
3,000080,NEBACETIN POWDER 5 G,"BACITRACIN,NEOMYCIN",pwd,NaN,NaN,3.468109e+11
4,000102,ALBOTHYL GEL 50 G,POLICRESULEN,gel,F,Oral Topical,2.660771e+08
...,...,...,...,...,...,...,...
593,033290,DISFLATYL TABLET,SIMETICONE,tab,A,Oral Solid,NaN
594,033322,VIRIDIS 500 MG TABLET,INOSINE PRANOBEX,tab,A,Oral Solid,NaN
595,033333,NYTEX 100MG/5ML DRY SYRUP 75 ML,ACETYLCYSTEINE,syr,B,Oral Liquid,NaN
596,033344,INHIPUMP,OMEPRAZOLE,NaN,NaN,NaN,NaN


In [88]:
def rekomendasi_produk(data_dokter, data_produk):
    hasil_rekomendasi = []

    for _, row in data_zat_aktif_dokter.iterrows():
        kode_rs = row['Kode RS']
        kode_dokter = row['Kode Dokter']
        kode_spesialis = row['Kode Spesialis']
        zat_aktif = row['Pack Molecule String']
        total = row['Total']

        # Filter produk yang memiliki zat aktif yang sama
        produk_cocok = data_produk[data_produk['Pack Molecule String'] == zat_aktif]

        if not produk_cocok.empty:
            # Filter berdasarkan bentuk sediaan
            produk_sediaan_cocok = produk_cocok[produk_cocok['Dosage Form'] == row['Dosage Form']]

            if not produk_sediaan_cocok.empty:
                # Jika ada yang cocok berdasarkan sediaan, ambil yang pertama
                produk_terpilih = produk_sediaan_cocok.iloc[0]
                metode_pemilihan = 'Bentuk Sediaan'
            else:
                # Jika tidak ada sediaan yang cocok, ambil produk dengan total sales tertinggi
                produk_terpilih = produk_cocok.sort_values(by='Total Penjualan', ascending=False).iloc[0]
                metode_pemilihan = 'Sales Tertinggi'

            hasil_rekomendasi.append({
                'kode_rs': kode_rs,
                'kode_dokter': kode_dokter,
                'kode_spesialis': kode_spesialis,
                'pack_molecule_string': zat_aktif,
                'produk_rekomendasi': produk_terpilih['Nama Produk'],
                'kode_produk': produk_terpilih['Kode Produk PI'],
                'metode_pemilihan': metode_pemilihan,
                'total': total
            })

    return pd.DataFrame(hasil_rekomendasi)

produk_rekomendasi = rekomendasi_produk(data_zat_aktif_dokter, data_pi_merged)
produk_rekomendasi

,kode_rs,kode_dokter,kode_spesialis,pack_molecule_string,produk_rekomendasi,kode_produk,metode_pemilihan,total
0,A1005681,A1000684,003,"BACITRACIN,NEOMYCIN",NEBACETIN POWDER 5 G,000080,Bentuk Sediaan,1.0
1,A1005681,A1000684,003,CEFADROXIL,DROVAX 500 KAPSUL,007527,Bentuk Sediaan,11.0
2,A1005681,A1000684,003,KETOROLAC,SCELTO 10 INJEKSI,007043,Bentuk Sediaan,1.0
3,A1005681,A1000684,003,KETOROLAC,SCELTO 30 INJEKSI,007054,Sales Tertinggi,10.0
4,A1005681,A1000684,003,LOPERAMIDE,RHOMUZ 2 MG FC TABLET,013038,Bentuk Sediaan,2.0
...,...,...,...,...,...,...,...,...
33447,I1001005,Q1002060,011,MELOXICAM,MOXAM 7.5 MG TABLET ASKES,012334,Bentuk Sediaan,80.0
33448,I1001005,Q1002060,011,METHYLPREDNISOLONE,TORAS 4 MG TABLET,008781,Bentuk Sediaan,4.0
33449,I1001005,Q1002060,011,OFLOXACIN,PHARFLOX 400 KAPLET FC,003952,Sales Tertinggi,17.0
33450,I1001005,Q1002060,011,PARACETAMOL,PRAXION 120MG/5ML SUSPENSI,007538,Sales Tertinggi,3.0


In [92]:
data_zat_aktif_dokter[(data_zat_aktif_dokter['Kode Dokter']=='A1000684') & (data_zat_aktif_dokter['Pack Molecule String'] == 'KETOROLAC')]

,Kode RS,Kode Dokter,Kode Spesialis,Pack Molecule String,Dosage Form,Total
11,A1005681,A1000684,003,KETOROLAC,inj,1.0
12,A1005681,A1000684,003,KETOROLAC,tab,10.0


In [93]:
data_survey_zat

,Tahun,Hospital Name,Doctor Name,Product Name without #,1,2,3,4,5,6,7,8,9,10,11,12,Pack Molecule String,Dosage Form
2,2020,RS Borromeus Bandung,APOTEKER,ABILIFY 5 MG TAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,30.0,ARIPIPRAZOLE,tab
4,2020,RS Borromeus Bandung,APOTEKER,ACANTHE SUNSCREEN SPF 30 CREAM 30GR,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,"ASCORBIC ACID,AVOBENZONE,CYCLOPENTASILOXANE,OC...",crm
7,2020,RS Borromeus Bandung,APOTEKER,ACETYLCYSTEINE 200 MG CAP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,ACETYLCYSTEINE,cap
8,2020,RS Borromeus Bandung,APOTEKER,ACETYLCYSTEINE 200MG/ML INF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ACETYLCYSTEINE,inf
9,2020,RS Borromeus Bandung,APOTEKER,ACID.SALICYL.CRYSTAL/POWDER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,51.0,"SALICYLIC ACID,TALC",pwd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659164,2024,RS Prima Medika,RSPM,ZOTER CREAM 5GR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,ACICLOVIR,crm
659165,2024,RS Prima Medika,RSPM,ZYCIN 500 MG TABLET,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,AZITHROMYCIN,tab
659166,2024,RS Prima Medika,RSPM,"ZYPRAZ 0,5 MG TAB",0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,ALPRAZOLAM,tab
659167,2024,RS Prima Medika,Tias Puspaningrum Amd Kep,ACETIN 200 MG KAP,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ACETYLCYSTEINE,cap
